# Criação de atributos temporais
Criação de atributos temporais a partir da agregação por períodos (dia, semana, mês, trimestre ou ano). Para cada atributo categórico no dataset, são gerados novos atributos considerando-se janelas móveis de três, seis e noves períodos consecutivos. 
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
O parâmetro `dataset` identifica os conjuntos de dados. Você pode importar arquivos de dataset com o botão  na barra de ferramentas.

In [ ]:
dataset = "/tmp/data/hotel_bookings.csv" #@param {type:"string"}
group_col = "hotel" #@param {type:"feature",label:"Atributo de agrupamento",description:"Atributo de agrupamento utilizado para a geração de atributos temporais."}
period = "mês" #@param ["dia","semana","mês","trimestre","ano"] {type:"string",multiple:false,label:"Período",description:"Período considerado para a geração de atributos temporais."}
date_col = "reservation_status_date" #@param {type:"feature",label:"Data de referência",description:"Atributo que determina a data de referência para a criação de atributos temporais"}
target_col = "reservation_status" #@param {type:"feature",label:"Atributo alvo",description:"O atributo alvo não pode ser considerado no processo de criação de novos atributos."}

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
import pandas as pd

data = pd.read_csv(dataset)

## Validação dos dados de entrada

In [ ]:
import platiagro
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)

numerical_cols = [
    metadata['columns'][i] 
    for i, ft in enumerate(metadata['featuretypes']) 
    if ft == platiagro.NUMERICAL]
numerical_cols = [col for col in numerical_cols if col != target_col]

categorical_cols = [
    metadata['columns'][i] 
    for i, ft in enumerate(metadata['featuretypes']) 
    if ft == platiagro.CATEGORICAL
]
categorical_cols = [col for col in categorical_cols if col != target_col]

datetime_cols = [
    metadata['columns'][i] 
    for i, ft in enumerate(metadata['featuretypes']) 
    if ft == platiagro.DATETIME
]


if len(numerical_cols) == 0:
    raise ValueError('Dataset deve necessariamente possuir um atributo do tipo numérico')

if group_col not in categorical_cols:
    raise ValueError('Atributo deve necessariamente ser do tipo categórico')
    
if date_col not in datetime_cols:
    raise ValueError('Atributo deve ser necessariamente do tipo datetime')

## Converte para o período selecionado o atributo referente à data 
A transformação é realizada pela função auxiliar `generate_new_index`. O perído obtido é então transformado em índice para faciliar a execução das próximas etapas.

In [ ]:
period_abbr = {
    'dia': 'D',
    'semana': 'W',
    'mês': 'M',
    'ano': 'Y',
    'trimestre': 'Q'
}

def generate_new_index(df: pd.DataFrame, date_col: str, period: str = 'mês'):
    if period not in period_abbr:
        raise KeyError(f'Parâmetro de entrada \'period\' precisa ser um dos seguintes: {list(period_abbr.keys())}.')
    return pd.DatetimeIndex(df[date_col]).to_period(period_abbr[period])

data[date_col] = pd.to_datetime(data[date_col])
data.index = generate_new_index(data, date_col, 'mês')

## Cálculo do quadrado dos atributos numéricos
Valores quadráticos serão usados no cálculo do desvio padrão móvel (`rolling std`).

In [ ]:
# for each target column calculates the square 
data = pd.concat([data, data[numerical_cols].pow(2).rename(columns=lambda x: 'SQR_' + x )], axis=1)

## Agregação e início do processo de criação de atributos temporais
Considerando-se os grupos determinados pelo período e atributo de agrupamento, são calculados para cada atributo numérico as seguintes medidas: `min`, `max`, `count` e `sum`.

In [ ]:
# aggregates the data by [date_col, group_col]
agg_functions = ['min', 'max', 'count', 'sum']
agg_df = data.groupby([data.index, group_col]).agg(
    {col : agg_functions for col in numerical_cols + ['SQR_' + col for col in numerical_cols]}
)

# fill missing (date_col, group_col) values
agg_df = agg_df.reindex(
    pd.MultiIndex.from_product(
        [agg_df.index.levels[0], agg_df.index.levels[1]], 
        names=[date_col, group_col]
    )
)
agg_df = agg_df.reset_index().sort_values([group_col, date_col],ignore_index=True)

## Função auxiliares para o cálculo dos atributos temporais
* média: `calculate_rolling_mean`
* min, max: `calculate_rolling_extrema`
* desvio padrão: `calculate_rolling_std`


In [ ]:
def calculate_rolling_mean(df: pd.DataFrame, target_cols, k: int = 3):
    agg_cols = [(col, f) for col in target_cols for f in ['count', 'sum']]
    res_df = df.groupby(group_col)[agg_cols].rolling(k, min_periods=1).sum()
    
    for col in target_cols:
        res_df[f'MEAN_{group_col}_{col}_{k}'] = res_df[(col, 'sum')] / res_df[(col, 'count')]
    
    res_df = res_df.drop(agg_cols, axis=1)
    res_df.columns = res_df.columns.droplevel(1) 
    return res_df

In [ ]:
def calculate_rolling_extrema(df: pd.DataFrame, target_cols, extrema: str = 'min', k: int = 3):
    agg_cols = [(col, extrema) for col in target_cols]
    if extrema == 'min':
        res_df = df.groupby(group_col)[agg_cols].rolling(k, min_periods=1).min().shift(1)
    else:
        res_df = df.groupby(group_col)[agg_cols].rolling(k, min_periods=1).max().shift(1)
    
    res_df.columns = res_df.columns.droplevel(1) 
    return res_df.rename(columns={col: f'{extrema.upper()}_{group_col}_{col}_{k}' for col in target_cols})  

In [ ]:
import numpy as np

def calculate_rolling_std(df: pd.DataFrame, target_cols, k:int = 3):
    agg_cols = [(col, f) for col in target_cols + ['SQR_' + col for col in target_cols] for f in ['count', 'sum']]
    res_df = df.groupby(group_col)[agg_cols].rolling(k, min_periods=1).sum().shift(1)
    
    for col in target_cols:
        new_name = f'STD_{group_col}_{col}_{k}'
        res_df[new_name] = res_df[('SQR_' + col, 'sum')] - (np.power(res_df[(col, 'sum')], 2) / res_df[(col, 'count')])
        res_df[new_name] = np.sqrt(res_df[new_name]) / (res_df[(col, 'count')] - 1)
        
    res_df = res_df.drop(agg_cols, axis=1)
    res_df.columns = res_df.columns.droplevel(1) 
    return res_df

## Cálculo dos atributos temporais

In [ ]:
new_features = pd.concat([
    pd.concat([calculate_rolling_mean(agg_df, numerical_cols, k) for k in [3, 6, 9]], axis=1),
    pd.concat([calculate_rolling_std(agg_df, numerical_cols, k) for k in [3, 6, 9]], axis=1),
    pd.concat([calculate_rolling_extrema(agg_df, numerical_cols, 'min', k) for k in [3, 6, 9]], axis=1),
    pd.concat([calculate_rolling_extrema(agg_df, numerical_cols, 'max', k) for k in [3, 6, 9]], axis=1)
], axis=1)
new_features[date_col] = agg_df[date_col].to_list()
new_features[group_col] = agg_df[group_col].to_list()

In [ ]:
#remove unnecessary columns
data.drop(['SQR_' + col for col in numerical_cols], axis=1, inplace=True)
#merge the generated features with the original data 
data.set_index(pd.Index(data[group_col]), append=True, inplace=True)
new_features.set_index([date_col, group_col], inplace=True)
data = pd.merge(data, new_features, left_index=True, right_index=True)

#reset index and sort values
data.set_index(pd.RangeIndex(start=0, stop=data.shape[0], step=1), inplace=True)

## Salva alterações no conjunto de dados

O conjunto de dados será salvo (e sobrescrito com as respectivas mudanças) localmente, no container da experimentação, utilizando a função `pandas.DataFrame.to_csv`.<br>

In [ ]:
data.to_csv(dataset, index=False)